# NIN网络

网络结构设计

In [36]:
from mxnet import gluon
from mxnet.gluon import nn
def nin_block(channels,kernel_size,padding,strides=1,maxpool=True):
    out = nn.Sequential()
    out.add(
        nn.Conv2D(channels,kernel_size=kernel_size,padding=padding,strides=strides,activation='relu'),
        nn.Conv2D(channels=channels, kernel_size=1,
                  padding=0, strides=1, activation='relu'),
        nn.Conv2D(channels=channels, kernel_size=1,
                  padding=0, strides=1, activation='relu'),
    )
    if maxpool==True:
        out.add(nn.MaxPool2D(pool_size=3,strides=2) )
    return out
def get_nin():
    net = nn.Sequential()
    with net.name_scope():
        net.add(
            nin_block(96,11,0,strides=4),
            nin_block(256,5,1),
            nin_block(384,3,1),
            nn.Dropout(.5),
            nin_block(10,3,1,maxpool=False),
            # 输入为 batch_size x 10 x 5 x 5, 通过AvgPool2D转成
            # batch_size x 10 x 1 x 1。
            nn.AvgPool2D(pool_size=5),
            # 转成 batch_size x 10
            nn.Flatten()
        )
    return net
    

In [33]:
from mxnet import nd
net = get_nin()
net.initialize()
data = nd.random.uniform(shape=(12,1,128,128))
y=net(data)
print(y.shape)

(12L, 10L, 1L, 1L)


In [ ]:
import sys
sys.path.append('..')
import utils
from mxnet import init
from mxnet import gluon

batch_size = 64
resize=224
train_data, test_data = utils.load_data_fashion_mnist(batch_size,resize=resize)
ctx = utils.try_gpu()

net=get_nin()
net.initialize(ctx=ctx,init=init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
#train(train_data, test_data, net, loss, trainer, ctx, num_epochs, print_batches=None):
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.1})
utils.train(train_data,test_data,net,loss,trainer,ctx,num_epochs=5,show=True)

('Start training on ', gpu(0))
Epoch 0. Loss: 2.303, Train acc 0.10, Test acc 0.10, Time 76.9 sec
